# RECOMMENDER SYSTEMM - UBCF
This project aims to randomly generate user-item matrix data and recommend a rating based on User-based Collaborative Filtering (UBCF)

**Input**: User-matrix item (with a blank rating to predict)\
**Output**: The rating for the empty node

## Step 1: Generate and initialize input user-item matrix

**User-item matrix**: As an example above an user-item matrix has this format:
|       | Item 1  | Item 2  | Item 3  |   ...   | Item n |
|:---   | :----:  | :----:  | :----:  |  :----: |   ---: |
|User 1 |    4    |    1    |    0    |   ...   |    5   |
|User 2 |    5    |    2    |    2    |   ...   |    3   |
|User 3 |    2    |    5    |    1    |   ...   |    1   |
|...    |   ...   |   ...   |   ...   |   ...   |   ...  |
|User m |    3    |    5    |    5    |   ...   |    4   |
Note: '0' value represents the empty value which needs to be predicted

The functions **empty_node, generate_user_item_matrix, devalue_matrix** serve the purposes to complete the generation of an user-item matrix

In [1]:
import numpy as np
import random
import copy
import time

In [2]:
start = time.time()

# This function generates a random position for the empty value in user-item matrix
def empty_node(num_row, num_col):
    """
    Generate a random position in a range between the start and stop numbers
    
    Args:
    start -- the beginning number
    end -- the ending number
    
    Returns:
    (row, col) -- a pair of generated row and column
    """
    row = random.randint(0, num_row - 1)
    col = random.randint(0, num_col - 1)
    return row, col

In [3]:
# This function generates a random user-item matrix
def generate_user_item_matrix(num_row, num_col):
    """
    It creates a matrix with 'num_row' rows and 'num_col' columns,
    whose values range between from 1 to 5 stars (inclusive)
    
    Args:
    num_row -- number of rows
    num_col -- number of columns
    
    Returns:
    np.ndarry -- the resulting numpy matrix
    """
    one_star = 1
    five_star = 5
        
    user_item_matrix = np.random.randint(one_star, five_star + 1, size=(num_row, num_col))
        
    return user_item_matrix

In [4]:
# This function empties a value in user-item matrix
def devalue_matrix(user_item_matrix, empty_node):
    """It devalues a value in user-item matrix, where the position is based on 'empty node' parameter
    
    Args:
    user_item_matrix -- the user-item matrix
    empty_node -- the provided position to devalue
    
    Returns:
    np.ndarray -- new user-item matrix with a '0' value
    """
    
    row, col = empty_node
    user_item_matrix[row, col] = 0
    return user_item_matrix

In [5]:
# This block is used as an example to test your result
# user_item_matrix = np.array([[5,4,1,4,0],
#                             [3,1,2,3,3],
#                             [4,3,4,3,5],
#                             [3,3,1,5,4],
#                             [1,5,5,2,1]])
# num_users = 5
# num_items = 5
# empty_node = (0, 4)
# Result: unrounded predicted rating = 4.24047254

## Step 2: Calculate Similarity values among Users

This step focuses on calculating the average rating of users' ratings, its variance, then the similarity of the neighborhoods with the user needs to be predicted for a item

**Step 2.1**: To calculate mean, the average rating values of users are computed by suming all ratings of a user then divide it by its number of ratings:
|       |  r_mean   |
|:---   |  :----:   |
|User 1 |    3.6    |
|User 2 |    2.7    |
|User 3 |    4.2    |
|...    |    ...    |
|User m |    2.9    |
Note: The number of ratings does not count '0' value(s)

In [6]:
# This function calculates mean of ratings among users
def calculate_mean(user_item_matrix, num_row):
    """ Calculate the mean of ratings among users,
    in which '0' values are ignored
    
    Args:
    user_item_matrix -- the input matrix data
    num_row -- number of rows (number of users)
    
    Returns:
    mean_rs -- the average array of ratings
    """
    mean_rs = []
    
    for i in range(num_row):
        # Get the ratings array of an user
        user_rating = user_item_matrix[i]
        
        # Get all values excluding zero numbers
        non_zero_values = user_rating[user_rating != 0]
        
        # Calculate mean
        mean = np.mean(non_zero_values) 
        mean_rs.append(mean)
    
    # Change to numpy array and reshape to (m, 1)
    mean_rs = np.array(mean_rs).reshape(num_row, 1)

    return mean_rs

**Step 2.2**: To calculate similarity, apply the operation for `PEARSON_CORRELATION`:
<center>
$ sim(u,v) = \frac{\sum{(r_{u,i} - \bar{r_u})(r_{v,i} - \bar{r_v})}}{\sqrt{\sum{(r_{u,i} - \bar{r_u})^2} \cdot \sum{(r_{v,i} - \bar{r_v})^2}}} $
</center>

In [7]:
# This function computes standard deviation of each user's rating
def std(user_item_matrix, mean_rs, num_users, num_items):
    """
    Args:
    user_item_matrix -- the input user-item matrix
    mean_rs -- the mean matrix of ratings
    num_users -- number of users
    num_items -- number of items
    
    Returns:
    std -- the standard deviation matrix of each user's ratings
    """
    stds = []
    
    for i in range(num_users):
        # Create an empty list to save intermediate values
        tmp_var = []
        
        # The 'count' parameter counts non-zero values in an array
        count = num_items
        
        # Get mean_r value of an user
        mean = mean_rs[i]
        
        # For each item in a list of user's rating
        for j in range(num_items):
            # Get r value of an item
            value = user_item_matrix[i][j]
            
            # If there's a 0 in the array, decrease the number of count by 1
            # Then ignore this element
            if value == 0:
                num_items -= 1
                continue
                
            # Compute the operation inside the square root
            tmp_var.append((value - mean) ** 2)
            
        # Sum all the values inside the square root, and get it squared root
        std = np.sqrt(np.sum(tmp_var))
        
        # Append the computed variance to the variance list
        stds.append(std)
        
    # Change the list to numpy array and reshape the variance list
    stds = np.array(stds).reshape(num_users, 1)
        
    return stds

In [8]:
# This function calculates the covariance of Pearson correlation coefficient (nominator of the expression)
def covariance(user_item_matrix, mean_rs, stds, num_users, num_items, empty_node):
    """
    Args:
    user_item_matrix -- the user-item matrix
    mean_rs -- the mean matrix of ratings
    stds -- the standard deviation matrix of ratings
    num_users -- the number of users
    num_items -- the number of items
    empty_node -- the position of the '0' value
    
    Returns:
    covariances -- the covariance matrix of each user's rating
    """
    covariances = []
    
    # Get the position of the empty node
    u_row, u_col = empty_node
    
    # Get r_u list and mean_r_u value
    r_u = user_item_matrix[u_row]
    mean_r_u = mean_rs[u_row][0]
    
    # For each user
    for i in range(num_users):
        # Create an empty list to save intermediate values
        tmp_var = []
        
        # Get r_v list and mean_r_v value
        r_v = user_item_matrix[i]
        mean_r_v = mean_rs[i][0]
        
        # If it reaches the position of the empty node (v == u)
        # Then assigns the covariance as NaN and skips to the next user 
        if i == u_row:
            covariances.append(np.nan)
            continue
            
        # For each item, compute multiplication list
        for j in range(num_items):
            # If there's an empty node in a column, then skips the item
            if j == u_col:
                continue
            
            # Compute multiplication (nominator)
            mul = (r_u[j] - mean_r_u) * (r_v[j] - mean_r_v)
            
            # Append the multiplication value to save in the list
            tmp_var.append(mul)
        
        # Sum all elements in the covariance list
        covariance = np.sum(tmp_var)
        
        # Add the computed covariance to the list of covariances of users
        covariances.append(covariance)
        
    # Change the list to numpy array and reshape the covariance list
    covariances = np.array(covariances).reshape(num_users, 1)
        
    return covariances

In [9]:
# This function computes the similarities between users
def similarity(user_item_matrix, covariances, stds, num_users, empty_node):
    """
    Args:
    user_item_matrix -- the user-item matrix
    covariances -- the arrays saving covariance values between users
    stds -- standard deviation values of each user
    num_users -- number of users
    empty_node -- the position of the '0' value
    
    Returns:
    similarity -- the similarity matrix
    """
    # Similarity list
    similarity = []
    
    # Get position of the u user and its standard deviation
    u_row, u_col = empty_node
    std_u = stds[u_row]
    
    # For each user v, calculate its similarity to u
    for i in range(num_users):
        # In case the user u is the same as v (v == u)
        # Then assigns the covariance as NaN and skips to the next user
        if i == u_row:
            similarity.append(-np.inf)
            continue
            
        # Get covariance and standard deviation values of neighbor user v
        cov_u_v = covariances[i]
        std_v = stds[i]
        
        # Compute the similarity value
        value = cov_u_v / (std_u * std_v)
        similarity.append(value[0])
        
    similarity = np.array(similarity).reshape(num_users, 1)
    
    return similarity

In [10]:
# This functions finds k-nearest neighbors to the user u
def k_nearest_neighbors(similarity, num_users, k):
    """
    Args: 
    similarity -- the similarity array between neighbor users to user u
    num_users -- number of users
    k -- the number to find k-largest similarity values
    
    Returns:
    most_similar -- k-size array of most similar neighbors to user u (a list of (similarity value, index))
    """
    most_similar = []
    
    # Copy the current similarity array
    sim = copy.deepcopy(similarity)
    
    # Reshape a (n, 1) array to (1, n) array
    sim = sim.reshape(1, num_users)
    
    # An array for intermediate calculations
    new_arr = []
    
    # For each usser
    for i in range(num_users):
        # Pair the similarity values and their index
        # Then assign it to a new array
        tmp = (sim[0][i], i)
        new_arr.append(tmp)

    # Sort the new array by their similarity values
    new_arr = sorted(new_arr)
    
    # Get the most similar values to the user u
    # in which the largest similarities are sorted ascendingly
    most_similar = new_arr[-k:]
    
    return most_similar

## Step 3: Predict

After getting all means, standard deviations, covariances, and similarities, let's predict the rating of user u for the empty item

In [11]:
# This function predicts the rating of user u for item i
def predict(user_item_matrix, most_similar, mean_rs, empty_node, k):
    """
    Args:
    user_item_matrix -- the user-item matrix
    most_similar -- k-size array of most similar neighbors to user u (a list of (similarity value, index))
    mean_rs -- the mean matrix of ratings
    empty_node -- the position of the '0' value
    k -- the number of k-nearest neighbors
    
    Returns:
    predicted_rating -- the predicting rating of user u for the item i
    """
    # Get the position of the '0' value
    u_row, u_col = empty_node
    
    # Get the mean value of user u
    mean_r_u = mean_rs[u_row]
    
    # Initialize numerator and denominator values of the prediction expression
    numerator = 0
    denominator = 0
    
    # For each nearest neighbor
    for i in range(k):
        # Get the similarity value and its index
        similarity, s_index = most_similar[i]
        
        # Get the rating of neighbor user v for item i (at u_col)
        r_v = user_item_matrix[s_index][u_col]
        
        # Get the mean rating of neighbor user v for item i (at u_col)
        mean_r_v = mean_rs[s_index]
        
        # Compute the numerator and denominator in the expression
        numerator += similarity * (r_v - mean_r_v)
        denominator += similarity
        
    # Compute the predicted rating of user u for item i    
    predicted_rating = mean_r_u + numerator / denominator
    return predicted_rating

In [12]:
# This main function runs kNN algorithm of UBCF to predict a user's rating
def kNN(num_users, num_items, u_row, u_col, k):
    """
    Args:
    num_users -- number of users
    num_items -- number of items
    u_row, u_col -- position of the empty node in user-item matrix
    k -- number of nearest neighbors
    
    Returns:
    user_item_matrix -- the user-item matrix
    predicted_rating -- the predicted rating of user u for item i
    (end - start) -- running time
    """
    start = time.time()
    
    # Generate user-item matrix
    user_item_matrix = generate_user_item_matrix(num_users, num_items)
    user_item_matrix = devalue_matrix(user_item_matrix, (u_row, u_col))
    
    # Get mean matrix
    mean_rs = calculate_mean(user_item_matrix, num_users)
    
    # Get standard deviation matrix of users' ratings
    stds = std(user_item_matrix, mean_rs, num_users, num_items)
    
    # Get covariance matrix of users' ratings
    covariances = covariance(user_item_matrix, mean_rs, stds, num_users, num_items, (u_row, u_col))

    # Get similarity matrix of users' ratings
    similarities = similarity(user_item_matrix, covariances, stds, num_users, (u_row, u_col))
    
    # Get k-nearest neighbors
    most_similar = k_nearest_neighbors(similarities, num_users, k)
    
    # Get the predicted rating of user u for item i 
    predicted_rating = predict(user_item_matrix, most_similar, mean_rs, (u_row, u_col), k)
    predicted_rating = int(predicted_rating)
    
    end = time.time()
    
    return user_item_matrix, predicted_rating, round((end - start), 2)

In [ ]:
# Initialize input data
num_users = random.randint(2, 10)
num_items = random.randint(2, 10)
k = random.randint(1, 10)

# Parameters for later plotting
num_users_list = []
num_items_list = []
running_time_list = []

# Run algorithm for a multiple times
for i in range(1, 10):
    # Exponentially increased matrix size
    num_users *= 2
    num_items *= 2
    
    # Running prediction
    u_row, u_col = empty_node(num_users, num_items)
    user_item_matrix, predicted_rating, running_time = kNN(num_users, num_items, u_row, u_col, k)
    
    # Print data
    print(f'{num_users} users. {num_items} items.')
    print(f'Predicted rating of User {u_row} for Item {u_col}: {predicted_rating} star(s). Running time: {running_time}s')
    
    # Save data for later plotting
    num_users_list.append(num_users)
    num_items_list.append(num_items)
    running_time_list.append(running_time)

8 users. 16 items.
Predicted rating of User 7 for Item 11: 4 star(s). Running time: 0.0s
16 users. 32 items.
Predicted rating of User 1 for Item 20: 2 star(s). Running time: 0.0s
32 users. 64 items.
Predicted rating of User 29 for Item 38: 2 star(s). Running time: 0.01s
64 users. 128 items.
Predicted rating of User 51 for Item 108: 3 star(s). Running time: 0.02s
128 users. 256 items.
Predicted rating of User 9 for Item 157: 3 star(s). Running time: 0.1s
256 users. 512 items.
Predicted rating of User 192 for Item 474: 1 star(s). Running time: 0.42s
512 users. 1024 items.
Predicted rating of User 46 for Item 812: 2 star(s). Running time: 2.07s
1024 users. 2048 items.
Predicted rating of User 61 for Item 1520: 2 star(s). Running time: 15.84s


In [ ]:
import matplotlib.pyplot as plt

num_users_list = np.array(num_users_list)
num_items_list = np.array(num_items_list)
running_time_list = np.array(running_time_list)

plt.figure(1)
plt.plot(num_users_list, label='Number of Users', color='red')
plt.plot(num_items_list, label='Number of Items', color='orange')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(2)
plt.plot(running_time_list, label='Running time', color='green')
plt.legend()
plt.grid(True)
plt.show()